<h1><align = center>Price Prediction of NYC Airbnb Rental Prices</align></h1>

## 1 Data Exploration

In [1]:
import numpy as np
import pandas as pd
import pixiedust
import pandas_profiling
pd.set_option('display.max_columns', None) 

Pixiedust database opened successfully


In [23]:
# read data
df = pd.read_csv("AB_NYC_2019.csv")
df.head()

,id,name,host_id,host_name,neighbourhood_group,neighbourhood,latitude,longitude,room_type,price,minimum_nights,number_of_reviews,last_review,reviews_per_month,calculated_host_listings_count,availability_365
0,2539,Clean & quiet apt home by the park,2787,John,Brooklyn,Kensington,40.64749,-73.97237,Private room,149,1,9,2018-10-19,0.21,6,365
1,2595,Skylit Midtown Castle,2845,Jennifer,Manhattan,Midtown,40.75362,-73.98377,Entire home/apt,225,1,45,2019-05-21,0.38,2,355
2,3647,THE VILLAGE OF HARLEM....NEW YORK !,4632,Elisabeth,Manhattan,Harlem,40.80902,-73.94190,Private room,150,3,0,NaN,NaN,1,365
3,3831,Cozy Entire Floor of Brownstone,4869,LisaRoxanne,Brooklyn,Clinton Hill,40.68514,-73.95976,Entire home/apt,89,1,270,2019-07-05,4.64,1,194
4,5022,Entire Apt: Spacious Studio/Loft by central park,7192,Laura,Manhattan,East Harlem,40.79851,-73.94399,Entire home/apt,80,10,9,2018-11-19,0.10,1,0


In [24]:
# formatting
df['id'] = df['id'].astype(str)
df['host_id'] = df['host_id'].astype(str)
df['last_review'] = pd.DatetimeIndex(df['last_review'])

In [25]:
df.columns

Index(['id', 'name', 'host_id', 'host_name', 'neighbourhood_group',
       'neighbourhood', 'latitude', 'longitude', 'room_type', 'price',
       'minimum_nights', 'number_of_reviews', 'last_review',
       'reviews_per_month', 'calculated_host_listings_count',
       'availability_365'],
      dtype='object')

In [26]:
df.shape

(48895, 16)

In [27]:
df.dtypes

id                                        object
name                                      object
host_id                                   object
host_name                                 object
neighbourhood_group                       object
neighbourhood                             object
latitude                                 float64
longitude                                float64
room_type                                 object
price                                      int64
minimum_nights                             int64
number_of_reviews                          int64
last_review                       datetime64[ns]
reviews_per_month                        float64
calculated_host_listings_count             int64
availability_365                           int64
dtype: object

In [8]:
profile = pandas_profiling.ProfileReport(df)
profile.to_file("nyc_ab_eda.html")

## Feature Engineering

In [31]:
# calculate days since last review
df['last_review'] = df['last_review'].fillna(np.min(df['last_review']))
df['days_since_last_review'] = (np.max(df['last_review']) - df['last_review']).dt.days

In [34]:
# one-hot encoding
room_dummies = pd.get_dummies(df['room_type'])
district_dummies = pd.get_dummies(df['neighbourhood_group'])
df = pd.concat([df, room_dummies, district_dummies], axis=1)
df.head()

,id,name,host_id,host_name,neighbourhood_group,neighbourhood,latitude,longitude,room_type,price,minimum_nights,number_of_reviews,last_review,reviews_per_month,calculated_host_listings_count,availability_365,days_since_last_review,Entire home/apt,Private room,Shared room,Bronx,Brooklyn,Manhattan,Queens,Staten Island
0,2539,Clean & quiet apt home by the park,2787,John,Brooklyn,Kensington,40.64749,-73.97237,Private room,149,1,9,2018-10-19,0.21,6,365,262,0,1,0,0,1,0,0,0
1,2595,Skylit Midtown Castle,2845,Jennifer,Manhattan,Midtown,40.75362,-73.98377,Entire home/apt,225,1,45,2019-05-21,0.38,2,355,48,1,0,0,0,0,1,0,0
2,3647,THE VILLAGE OF HARLEM....NEW YORK !,4632,Elisabeth,Manhattan,Harlem,40.80902,-73.94190,Private room,150,3,0,2011-03-28,NaN,1,365,3024,0,1,0,0,0,1,0,0
3,3831,Cozy Entire Floor of Brownstone,4869,LisaRoxanne,Brooklyn,Clinton Hill,40.68514,-73.95976,Entire home/apt,89,1,270,2019-07-05,4.64,1,194,3,1,0,0,0,1,0,0,0
4,5022,Entire Apt: Spacious Studio/Loft by central park,7192,Laura,Manhattan,East Harlem,40.79851,-73.94399,Entire home/apt,80,10,9,2018-11-19,0.10,1,0,231,1,0,0,0,0,1,0,0


In [47]:
# gender of host
import spacy
nlp = spacy.load('en_core_web_sm')

female = nlp('female')
male = nlp('male')

gender = []

i = 0
for name in df['host_name']:
    i += 1
    print(i, end=' ')
    if type(name) == str:
        female_score = female.similarity(nlp(name))
        male_score = male.similarity(nlp(name))
        gen = 'female' if female_score >= male_score else 'male'
        gender.append(gen)
    else:
        gender.append('unknown')
df['host_gender'] = gender

1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18 19 20 21 22 23 24 25 26 27 28 29 30 31 32 33 34 35 36 37 38 39 40 41 42 43 44 45 46 47 48 49 50 51 52 53 54 55 56 57 58 59 60 61 62 63 64 65 66 67 68 69 70 71 72 73 74 75 76 77 78 79 80 81 82 83 84 85 86 87 88 89 90 91 92 93 94 95 96 97 98 99 100 101 102 103 104 105 106 107 108 109 110 111 112 113 114 115 116 117 118 119 120 121 122 123 124 125 126 127 128 129 130 131 132 133 134 135 136 137 138 139 140 141 142 143 144 145 146 147 148 149 150 151 152 153 154 155 156 157 158 159 160 161 162 163 164 165 166 167 168 169 170 171 172 173 174 175 176 177 178 179 180 181 182 183 184 185 186 187 188 189 190 191 192 193 194 195 196 197 198 199 200 201 202 203 204 205 206 207 208 209 210 211 212 213 214 215 216 217 218 219 220 221 222 223 224 225 226 227 228 229 230 231 232 233 234 235 236 237 238 239 240 241 242 243 244 245 246 247 248 249 250 251 252 253 254 255 256 257 258 259 260 261 262 263 264 265 266 267 268 269 270 271 272 273 274 275 276 277 

1861 1862 1863 1864 1865 1866 1867 1868 1869 1870 1871 1872 1873 1874 1875 1876 1877 1878 1879 1880 1881 1882 1883 1884 1885 1886 1887 1888 1889 1890 1891 1892 1893 1894 1895 1896 1897 1898 1899 1900 1901 1902 1903 1904 1905 1906 1907 1908 1909 1910 1911 1912 1913 1914 1915 1916 1917 1918 1919 1920 1921 1922 1923 1924 1925 1926 1927 1928 1929 1930 1931 1932 1933 1934 1935 1936 1937 1938 1939 1940 1941 1942 1943 1944 1945 1946 1947 1948 1949 1950 1951 1952 1953 1954 1955 1956 1957 1958 1959 1960 1961 1962 1963 1964 1965 1966 1967 1968 1969 1970 1971 1972 1973 1974 1975 1976 1977 1978 1979 1980 1981 1982 1983 1984 1985 1986 1987 1988 1989 1990 1991 1992 1993 1994 1995 1996 1997 1998 1999 2000 2001 2002 2003 2004 2005 2006 2007 2008 2009 2010 2011 2012 2013 2014 2015 2016 2017 2018 2019 2020 2021 2022 2023 2024 2025 2026 2027 2028 2029 2030 2031 2032 2033 2034 2035 2036 2037 2038 2039 2040 2041 2042 2043 2044 2045 2046 2047 2048 2049 2050 2051 2052 2053 2054 2055 2056 2057 2058 2059 2060 

 3500 3501 3502 3503 3504 3505 3506 3507 3508 3509 3510 3511 3512 3513 3514 3515 3516 3517 3518 3519 3520 3521 3522 3523 3524 3525 3526 3527 3528 3529 3530 3531 3532 3533 3534 3535 3536 3537 3538 3539 3540 3541 3542 3543 3544 3545 3546 3547 3548 3549 3550 3551 3552 3553 3554 3555 3556 3557 3558 3559 3560 3561 3562 3563 3564 3565 3566 3567 3568 3569 3570 3571 3572 3573 3574 3575 3576 3577 3578 3579 3580 3581 3582 3583 3584 3585 3586 3587 3588 3589 3590 3591 3592 3593 3594 3595 3596 3597 3598 3599 3600 3601 3602 3603 3604 3605 3606 3607 3608 3609 3610 3611 3612 3613 3614 3615 3616 3617 3618 3619 3620 3621 3622 3623 3624 3625 3626 3627 3628 3629 3630 3631 3632 3633 3634 3635 3636 3637 3638 3639 3640 3641 3642 3643 3644 3645 3646 3647 3648 3649 3650 3651 3652 3653 3654 3655 3656 3657 3658 3659 3660 3661 3662 3663 3664 3665 3666 3667 3668 3669 3670 3671 3672 3673 3674 3675 3676 3677 3678 3679 3680 3681 3682 3683 3684 3685 3686 3687 3688 3689 3690 3691 3692 3693 3694 3695 3696 3697 3698 3699

 5139 5140 5141 5142 5143 5144 5145 5146 5147 5148 5149 5150 5151 5152 5153 5154 5155 5156 5157 5158 5159 5160 5161 5162 5163 5164 5165 5166 5167 5168 5169 5170 5171 5172 5173 5174 5175 5176 5177 5178 5179 5180 5181 5182 5183 5184 5185 5186 5187 5188 5189 5190 5191 5192 5193 5194 5195 5196 5197 5198 5199 5200 5201 5202 5203 5204 5205 5206 5207 5208 5209 5210 5211 5212 5213 5214 5215 5216 5217 5218 5219 5220 5221 5222 5223 5224 5225 5226 5227 5228 5229 5230 5231 5232 5233 5234 5235 5236 5237 5238 5239 5240 5241 5242 5243 5244 5245 5246 5247 5248 5249 5250 5251 5252 5253 5254 5255 5256 5257 5258 5259 5260 5261 5262 5263 5264 5265 5266 5267 5268 5269 5270 5271 5272 5273 5274 5275 5276 5277 5278 5279 5280 5281 5282 5283 5284 5285 5286 5287 5288 5289 5290 5291 5292 5293 5294 5295 5296 5297 5298 5299 5300 5301 5302 5303 5304 5305 5306 5307 5308 5309 5310 5311 5312 5313 5314 5315 5316 5317 5318 5319 5320 5321 5322 5323 5324 5325 5326 5327 5328 5329 5330 5331 5332 5333 5334 5335 5336 5337 5338

 6778 6779 6780 6781 6782 6783 6784 6785 6786 6787 6788 6789 6790 6791 6792 6793 6794 6795 6796 6797 6798 6799 6800 6801 6802 6803 6804 6805 6806 6807 6808 6809 6810 6811 6812 6813 6814 6815 6816 6817 6818 6819 6820 6821 6822 6823 6824 6825 6826 6827 6828 6829 6830 6831 6832 6833 6834 6835 6836 6837 6838 6839 6840 6841 6842 6843 6844 6845 6846 6847 6848 6849 6850 6851 6852 6853 6854 6855 6856 6857 6858 6859 6860 6861 6862 6863 6864 6865 6866 6867 6868 6869 6870 6871 6872 6873 6874 6875 6876 6877 6878 6879 6880 6881 6882 6883 6884 6885 6886 6887 6888 6889 6890 6891 6892 6893 6894 6895 6896 6897 6898 6899 6900 6901 6902 6903 6904 6905 6906 6907 6908 6909 6910 6911 6912 6913 6914 6915 6916 6917 6918 6919 6920 6921 6922 6923 6924 6925 6926 6927 6928 6929 6930 6931 6932 6933 6934 6935 6936 6937 6938 6939 6940 6941 6942 6943 6944 6945 6946 6947 6948 6949 6950 6951 6952 6953 6954 6955 6956 6957 6958 6959 6960 6961 6962 6963 6964 6965 6966 6967 6968 6969 6970 6971 6972 6973 6974 6975 6976 6977

...

 42831 42832 42833 42834 42835 42836 42837 42838 42839 42840 42841 42842 42843 42844 42845 42846 42847 42848 42849 42850 42851 42852 42853 42854 42855 42856 42857 42858 42859 42860 42861 42862 42863 42864 42865 42866 42867 42868 42869 42870 42871 42872 42873 42874 42875 42876 42877 42878 42879 42880 42881 42882 42883 42884 42885 42886 42887 42888 42889 42890 42891 42892 42893 42894 42895 42896 42897 42898 42899 42900 42901 42902 42903 42904 42905 42906 42907 42908 42909 42910 42911 42912 42913 42914 42915 42916 42917 42918 42919 42920 42921 42922 42923 42924 42925 42926 42927 42928 42929 42930 42931 42932 42933 42934 42935 42936 42937 42938 42939 42940 42941 42942 42943 42944 42945 42946 42947 42948 42949 42950 42951 42952 42953 42954 42955 42956 42957 42958 42959 42960 42961 42962 42963 42964 42965 42966 42967 42968 42969 42970 42971 42972 42973 42974 42975 42976 42977 42978 42979 42980 42981 42982 42983 42984 42985 42986 42987 42988 42989 42990 42991 42992 42993 42994 42995 42996 429

 44197 44198 44199 44200 44201 44202 44203 44204 44205 44206 44207 44208 44209 44210 44211 44212 44213 44214 44215 44216 44217 44218 44219 44220 44221 44222 44223 44224 44225 44226 44227 44228 44229 44230 44231 44232 44233 44234 44235 44236 44237 44238 44239 44240 44241 44242 44243 44244 44245 44246 44247 44248 44249 44250 44251 44252 44253 44254 44255 44256 44257 44258 44259 44260 44261 44262 44263 44264 44265 44266 44267 44268 44269 44270 44271 44272 44273 44274 44275 44276 44277 44278 44279 44280 44281 44282 44283 44284 44285 44286 44287 44288 44289 44290 44291 44292 44293 44294 44295 44296 44297 44298 44299 44300 44301 44302 44303 44304 44305 44306 44307 44308 44309 44310 44311 44312 44313 44314 44315 44316 44317 44318 44319 44320 44321 44322 44323 44324 44325 44326 44327 44328 44329 44330 44331 44332 44333 44334 44335 44336 44337 44338 44339 44340 44341 44342 44343 44344 44345 44346 44347 44348 44349 44350 44351 44352 44353 44354 44355 44356 44357 44358 44359 44360 44361 44362 443

 45563 45564 45565 45566 45567 45568 45569 45570 45571 45572 45573 45574 45575 45576 45577 45578 45579 45580 45581 45582 45583 45584 45585 45586 45587 45588 45589 45590 45591 45592 45593 45594 45595 45596 45597 45598 45599 45600 45601 45602 45603 45604 45605 45606 45607 45608 45609 45610 45611 45612 45613 45614 45615 45616 45617 45618 45619 45620 45621 45622 45623 45624 45625 45626 45627 45628 45629 45630 45631 45632 45633 45634 45635 45636 45637 45638 45639 45640 45641 45642 45643 45644 45645 45646 45647 45648 45649 45650 45651 45652 45653 45654 45655 45656 45657 45658 45659 45660 45661 45662 45663 45664 45665 45666 45667 45668 45669 45670 45671 45672 45673 45674 45675 45676 45677 45678 45679 45680 45681 45682 45683 45684 45685 45686 45687 45688 45689 45690 45691 45692 45693 45694 45695 45696 45697 45698 45699 45700 45701 45702 45703 45704 45705 45706 45707 45708 45709 45710 45711 45712 45713 45714 45715 45716 45717 45718 45719 45720 45721 45722 45723 45724 45725 45726 45727 45728 457

 46929 46930 46931 46932 46933 46934 46935 46936 46937 46938 46939 46940 46941 46942 46943 46944 46945 46946 46947 46948 46949 46950 46951 46952 46953 46954 46955 46956 46957 46958 46959 46960 46961 46962 46963 46964 46965 46966 46967 46968 46969 46970 46971 46972 46973 46974 46975 46976 46977 46978 46979 46980 46981 46982 46983 46984 46985 46986 46987 46988 46989 46990 46991 46992 46993 46994 46995 46996 46997 46998 46999 47000 47001 47002 47003 47004 47005 47006 47007 47008 47009 47010 47011 47012 47013 47014 47015 47016 47017 47018 47019 47020 47021 47022 47023 47024 47025 47026 47027 47028 47029 47030 47031 47032 47033 47034 47035 47036 47037 47038 47039 47040 47041 47042 47043 47044 47045 47046 47047 47048 47049 47050 47051 47052 47053 47054 47055 47056 47057 47058 47059 47060 47061 47062 47063 47064 47065 47066 47067 47068 47069 47070 47071 47072 47073 47074 47075 47076 47077 47078 47079 47080 47081 47082 47083 47084 47085 47086 47087 47088 47089 47090 47091 47092 47093 47094 470

 48295 48296 48297 48298 48299 48300 48301 48302 48303 48304 48305 48306 48307 48308 48309 48310 48311 48312 48313 48314 48315 48316 48317 48318 48319 48320 48321 48322 48323 48324 48325 48326 48327 48328 48329 48330 48331 48332 48333 48334 48335 48336 48337 48338 48339 48340 48341 48342 48343 48344 48345 48346 48347 48348 48349 48350 48351 48352 48353 48354 48355 48356 48357 48358 48359 48360 48361 48362 48363 48364 48365 48366 48367 48368 48369 48370 48371 48372 48373 48374 48375 48376 48377 48378 48379 48380 48381 48382 48383 48384 48385 48386 48387 48388 48389 48390 48391 48392 48393 48394 48395 48396 48397 48398 48399 48400 48401 48402 48403 48404 48405 48406 48407 48408 48409 48410 48411 48412 48413 48414 48415 48416 48417 48418 48419 48420 48421 48422 48423 48424 48425 48426 48427 48428 48429 48430 48431 48432 48433 48434 48435 48436 48437 48438 48439 48440 48441 48442 48443 48444 48445 48446 48447 48448 48449 48450 48451 48452 48453 48454 48455 48456 48457 48458 48459 48460 484

In [49]:
gender_dummies = pd.get_dummies(df['host_gender'])
df = pd.concat([df, gender_dummies], axis=1)

In [60]:
df.head()

,id,name,host_id,host_name,neighbourhood_group,neighbourhood,latitude,longitude,room_type,price,minimum_nights,number_of_reviews,last_review,reviews_per_month,calculated_host_listings_count,availability_365,days_since_last_review,Entire home/apt,Private room,Shared room,Bronx,Brooklyn,Manhattan,Queens,Staten Island,host_gender,female,male,unknown
0,2539,Clean & quiet apt home by the park,2787,John,Brooklyn,Kensington,40.64749,-73.97237,Private room,149,1,9,2018-10-19,0.21,6,365,262,0,1,0,0,1,0,0,0,male,0,1,0
1,2595,Skylit Midtown Castle,2845,Jennifer,Manhattan,Midtown,40.75362,-73.98377,Entire home/apt,225,1,45,2019-05-21,0.38,2,355,48,1,0,0,0,0,1,0,0,female,1,0,0
2,3647,THE VILLAGE OF HARLEM....NEW YORK !,4632,Elisabeth,Manhattan,Harlem,40.80902,-73.94190,Private room,150,3,0,2011-03-28,NaN,1,365,3024,0,1,0,0,0,1,0,0,male,0,1,0
3,3831,Cozy Entire Floor of Brownstone,4869,LisaRoxanne,Brooklyn,Clinton Hill,40.68514,-73.95976,Entire home/apt,89,1,270,2019-07-05,4.64,1,194,3,1,0,0,0,1,0,0,0,female,1,0,0
4,5022,Entire Apt: Spacious Studio/Loft by central park,7192,Laura,Manhattan,East Harlem,40.79851,-73.94399,Entire home/apt,80,10,9,2018-11-19,0.10,1,0,231,1,0,0,0,0,1,0,0,female,1,0,0


In [63]:
y = df['price']
df.drop(['room_type', 'last_review'], axis=1, inplace=True)

In [65]:
df['reviews_per_month'] = df['reviews_per_month'].fillna(0)
df.drop(['host_gender'], axis=1, inplace=True)

In [66]:
df.head()

,id,name,host_id,host_name,neighbourhood_group,neighbourhood,latitude,longitude,price,minimum_nights,number_of_reviews,reviews_per_month,calculated_host_listings_count,availability_365,days_since_last_review,Entire home/apt,Private room,Shared room,Bronx,Brooklyn,Manhattan,Queens,Staten Island,female,male,unknown
0,2539,Clean & quiet apt home by the park,2787,John,Brooklyn,Kensington,40.64749,-73.97237,149,1,9,0.21,6,365,262,0,1,0,0,1,0,0,0,0,1,0
1,2595,Skylit Midtown Castle,2845,Jennifer,Manhattan,Midtown,40.75362,-73.98377,225,1,45,0.38,2,355,48,1,0,0,0,0,1,0,0,1,0,0
2,3647,THE VILLAGE OF HARLEM....NEW YORK !,4632,Elisabeth,Manhattan,Harlem,40.80902,-73.94190,150,3,0,0.00,1,365,3024,0,1,0,0,0,1,0,0,0,1,0
3,3831,Cozy Entire Floor of Brownstone,4869,LisaRoxanne,Brooklyn,Clinton Hill,40.68514,-73.95976,89,1,270,4.64,1,194,3,1,0,0,0,1,0,0,0,1,0,0
4,5022,Entire Apt: Spacious Studio/Loft by central park,7192,Laura,Manhattan,East Harlem,40.79851,-73.94399,80,10,9,0.10,1,0,231,1,0,0,0,0,1,0,0,1,0,0


In [67]:
df.to_csv("df.csv")

## Setting up ML Model

In [94]:
X = pd.concat([df.iloc[:,6:8], df.iloc[:,9:]], axis=1)
y = df['price']
X.head()

,latitude,longitude,minimum_nights,number_of_reviews,reviews_per_month,calculated_host_listings_count,availability_365,days_since_last_review,Entire home/apt,Private room,Shared room,Bronx,Brooklyn,Manhattan,Queens,Staten Island,female,male,unknown
0,40.64749,-73.97237,1,9,0.21,6,365,262,0,1,0,0,1,0,0,0,0,1,0
1,40.75362,-73.98377,1,45,0.38,2,355,48,1,0,0,0,0,1,0,0,1,0,0
2,40.80902,-73.94190,3,0,0.00,1,365,3024,0,1,0,0,0,1,0,0,0,1,0
3,40.68514,-73.95976,1,270,4.64,1,194,3,1,0,0,0,1,0,0,0,1,0,0
4,40.79851,-73.94399,10,9,0.10,1,0,231,1,0,0,0,0,1,0,0,1,0,0


In [95]:
print(X.shape, y.shape)

(48895, 19) (48895,)


In [107]:
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LinearRegression, Lasso, Ridge
from sklearn.ensemble import RandomForestRegressor

from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import mean_squared_error

X_train, X_test, y_train, y_test = train_test_split(X, y)

scaler = StandardScaler()
lm = LinearRegression()
lasso = Lasso()
ridge = Ridge()
rf = RandomForestRegressor()
pca = PCA()

In [110]:
regressors = [lm, lasso, ridge, rf]
for reg in regressors:
    pipe = Pipeline([
        ('scale', scaler),
        ('pca', pca),
        ('reg', reg)
    ])
    pipe.fit(X_train, y_train)
    y_pred = pipe.predict(X_test)
    rmse = np.sqrt(mean_squared_error(y_test, y_pred))
    print('{},{}'.format(reg, rmse))

LinearRegression(copy_X=True, fit_intercept=True, n_jobs=None, normalize=False),213.18871540866832
Lasso(alpha=1.0, copy_X=True, fit_intercept=True, max_iter=1000,
      normalize=False, positive=False, precompute=False, random_state=None,
      selection='cyclic', tol=0.0001, warm_start=False),213.16668009257953
Ridge(alpha=1.0, copy_X=True, fit_intercept=True, max_iter=None,
      normalize=False, random_state=None, solver='auto', tol=0.001),213.20361194082835
RandomForestRegressor(bootstrap=True, criterion='mse', max_depth=None,
                      max_features='auto', max_leaf_nodes=None,
                      min_impurity_decrease=0.0, min_impurity_split=None,
                      min_samples_leaf=1, min_samples_split=2,
                      min_weight_fraction_leaf=0.0, n_estimators=10,
                      n_jobs=None, oob_score=False, random_state=None,
                      verbose=0, warm_start=False),220.82153056353064


In [111]:
# without PCA

In [117]:
# best: Lasso
regressors = [lm, lasso, ridge, rf]
for reg in regressors:
    pipe = Pipeline([
        ('scale', scaler),
        ('reg', reg)
    ])
    pipe.fit(X_train, y_train)
    y_pred = pipe.predict(X_test)
    rmse = np.sqrt(mean_squared_error(y_test, y_pred))
    print('{},{}'.format(reg, rmse))

LinearRegression(copy_X=True, fit_intercept=True, n_jobs=None, normalize=False),213.1868702496031
Lasso(alpha=1.0, copy_X=True, fit_intercept=True, max_iter=1000,
      normalize=False, positive=False, precompute=False, random_state=None,
      selection='cyclic', tol=0.0001, warm_start=False),213.15058532806322
Ridge(alpha=1.0, copy_X=True, fit_intercept=True, max_iter=None,
      normalize=False, random_state=None, solver='auto', tol=0.001),213.20361194082835
RandomForestRegressor(bootstrap=True, criterion='mse', max_depth=None,
                      max_features='auto', max_leaf_nodes=None,
                      min_impurity_decrease=0.0, min_impurity_split=None,
                      min_samples_leaf=1, min_samples_split=2,
                      min_weight_fraction_leaf=0.0, n_estimators=10,
                      n_jobs=None, oob_score=False, random_state=None,
                      verbose=0, warm_start=False),220.9318998101686


Try adding polynomial features.

In [118]:
from sklearn.preprocessing import PolynomialFeatures
poly = PolynomialFeatures(degree=2)

In [120]:
regressors = [lm, lasso, ridge, rf]
for reg in regressors:
    pipe = Pipeline([
        ('poly', poly),
        ('scale', scaler),
        ('reg', reg)
    ])
    pipe.fit(X_train, y_train)
    y_pred = pipe.predict(X_test)
    rmse = np.sqrt(mean_squared_error(y_test, y_pred))
    print('{},{}'.format(reg, rmse))
# rmse explosion
# we won't be using polynomial features

LinearRegression(copy_X=True, fit_intercept=True, n_jobs=None, normalize=False),17326594478873.307


KeyboardInterrupt: 

Try adding 'name' by converting it to word embedding.

In [144]:
full_vec = np.array([])
i = 0
for name in df['name']:
    i += 1
    print(i, end=' ')
    vec = nlp(name).vector
    full_vec = np.hstack((full_vec, vec))

1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18 19 20 21 22 23 24 25 26 27 28 29 30 31 32 33 34 35 36 37 38 39 40 41 42 43 44 45 46 47 48 49 50 51 52 53 54 55 56 57 58 59 60 61 62 63 64 65 66 67 68 69 70 71 72 73 74 75 76 77 78 79 80 81 82 83 84 85 86 87 88 89 90 91 92 93 94 95 96 97 98 99 100 101 102 103 104 105 106 107 108 109 110 111 112 113 114 115 116 117 118 119 120 121 122 123 124 125 126 127 128 129 130 131 132 133 134 135 136 137 138 139 140 141 142 143 144 145 146 147 148 149 150 151 152 153 154 155 156 157 158 159 160 161 162 163 164 165 166 167 168 169 170 171 172 173 174 175 176 177 178 179 180 181 182 183 184 185 186 187 188 189 190 191 192 193 194 195 196 197 198 199 200 201 202 203 204 205 206 207 208 209 210 211 212 213 214 215 216 217 218 219 220 221 222 223 224 225 226 227 228 229 230 231 232 233 234 235 236 237 238 239 240 241 242 243 244 245 246 247 248 249 250 251 252 253 254 255 256 257 258 259 260 261 262 263 264 265 266 267 268 269 270 271 272 273 274 275 276 277 

1861 1862 1863 1864 1865 1866 1867 1868 1869 1870 1871 1872 1873 1874 1875 1876 1877 1878 1879 1880 1881 1882 1883 1884 1885 1886 1887 1888 1889 1890 1891 1892 1893 1894 1895 1896 1897 1898 1899 1900 1901 1902 1903 1904 1905 1906 1907 1908 1909 1910 1911 1912 1913 1914 1915 1916 1917 1918 1919 1920 1921 1922 1923 1924 1925 1926 1927 1928 1929 1930 1931 1932 1933 1934 1935 1936 1937 1938 1939 1940 1941 1942 1943 1944 1945 1946 1947 1948 1949 1950 1951 1952 1953 1954 1955 1956 1957 1958 1959 1960 1961 1962 1963 1964 1965 1966 1967 1968 1969 1970 1971 1972 1973 1974 1975 1976 1977 1978 1979 1980 1981 1982 1983 1984 1985 1986 1987 1988 1989 1990 1991 1992 1993 1994 1995 1996 1997 1998 1999 2000 2001 2002 2003 2004 2005 2006 2007 2008 2009 2010 2011 2012 2013 2014 2015 2016 2017 2018 2019 2020 2021 2022 2023 2024 2025 2026 2027 2028 2029 2030 2031 2032 2033 2034 2035 2036 2037 2038 2039 2040 2041 2042 2043 2044 2045 2046 2047 2048 2049 2050 2051 2052 2053 2054 2055 2056 2057 2058 2059 2060 

 3500 3501 3502 3503 3504 3505 3506 3507 3508 3509 3510 3511 3512 3513 3514 3515 3516 3517 3518 3519 3520 3521 3522 3523 3524 3525 3526 3527 3528 3529 3530 3531 3532 3533 3534 3535 3536 3537 3538 3539 3540 3541 3542 3543 3544 3545 3546 3547 3548 3549 3550 3551 3552 3553 3554 3555 3556 3557 3558 3559 3560 3561 3562 3563 3564 3565 3566 3567 3568 3569 3570 3571 3572 3573 3574 3575 3576 3577 3578 3579 3580 3581 3582 3583 3584 3585 3586 3587 3588 3589 3590 3591 3592 3593 3594 3595 3596 3597 3598 3599 3600 3601 3602 3603 3604 3605 3606 3607 3608 3609 3610 3611 3612 3613 3614 3615 3616 3617 3618 3619 3620 3621 3622 3623 3624 3625 3626 3627 3628 3629 3630 3631 3632 3633 3634 3635 3636 3637 3638 3639 3640 3641 3642 3643 3644 3645 3646 3647 3648 3649 3650 3651 3652 3653 3654 3655 3656 3657 3658 3659 3660 3661 3662 3663 3664 3665 3666 3667 3668 3669 3670 3671 3672 3673 3674 3675 3676 3677 3678 3679 3680 3681 3682 3683 3684 3685 3686 3687 3688 3689 3690 3691 3692 3693 3694 3695 3696 3697 3698 3699

 5139 5140 5141 5142 5143 5144 5145 5146 5147 5148 5149 5150 5151 5152 5153 5154 5155 5156 5157 5158 5159 5160 5161 5162 5163 5164 5165 5166 5167 5168 5169 5170 5171 5172 5173 5174 5175 5176 5177 5178 5179 5180 5181 5182 5183 5184 5185 5186 5187 5188 5189 5190 5191 5192 5193 5194 5195 5196 5197 5198 5199 5200 5201 5202 5203 5204 5205 5206 5207 5208 5209 5210 5211 5212 5213 5214 5215 5216 5217 5218 5219 5220 5221 5222 5223 5224 5225 5226 5227 5228 5229 5230 5231 5232 5233 5234 5235 5236 5237 5238 5239 5240 5241 5242 5243 5244 5245 5246 5247 5248 5249 5250 5251 5252 5253 5254 5255 5256 5257 5258 5259 5260 5261 5262 5263 5264 5265 5266 5267 5268 5269 5270 5271 5272 5273 5274 5275 5276 5277 5278 5279 5280 5281 5282 5283 5284 5285 5286 5287 5288 5289 5290 5291 5292 5293 5294 5295 5296 5297 5298 5299 5300 5301 5302 5303 5304 5305 5306 5307 5308 5309 5310 5311 5312 5313 5314 5315 5316 5317 5318 5319 5320 5321 5322 5323 5324 5325 5326 5327 5328 5329 5330 5331 5332 5333 5334 5335 5336 5337 5338

 6778 6779 6780 6781 6782 6783 6784 6785 6786 6787 6788 6789 6790 6791 6792 6793 6794 6795 6796 6797 6798 6799 6800 6801 6802 6803 6804 6805 6806 6807 6808 6809 6810 6811 6812 6813 6814 6815 6816 6817 6818 6819 6820 6821 6822 6823 6824 6825 6826 6827 6828 6829 6830 6831 6832 6833 6834 6835 6836 6837 6838 6839 6840 6841 6842 6843 6844 6845 6846 6847 6848 6849 6850 6851 6852 6853 6854 6855 6856 6857 6858 6859 6860 6861 6862 6863 6864 6865 6866 6867 6868 6869 6870 6871 6872 6873 6874 6875 6876 6877 6878 6879 6880 6881 6882 6883 6884 6885 6886 6887 6888 6889 6890 6891 6892 6893 6894 6895 6896 6897 6898 6899 6900 6901 6902 6903 6904 6905 6906 6907 6908 6909 6910 6911 6912 6913 6914 6915 6916 6917 6918 6919 6920 6921 6922 6923 6924 6925 6926 6927 6928 6929 6930 6931 6932 6933 6934 6935 6936 6937 6938 6939 6940 6941 6942 6943 6944 6945 6946 6947 6948 6949 6950 6951 6952 6953 6954 6955 6956 6957 6958 6959 6960 6961 6962 6963 6964 6965 6966 6967 6968 6969 6970 6971 6972 6973 6974 6975 6976 6977

...

 42831 42832 42833 42834 42835 42836 42837 42838 42839 42840 42841 42842 42843 42844 42845 42846 42847 42848 42849 42850 42851 42852 42853 42854 42855 42856 42857 42858 42859 42860 42861 42862 42863 42864 42865 42866 42867 42868 42869 42870 42871 42872 42873 42874 42875 42876 42877 42878 42879 42880 42881 42882 42883 42884 42885 42886 42887 42888 42889 42890 42891 42892 42893 42894 42895 42896 42897 42898 42899 42900 42901 42902 42903 42904 42905 42906 42907 42908 42909 42910 42911 42912 42913 42914 42915 42916 42917 42918 42919 42920 42921 42922 42923 42924 42925 42926 42927 42928 42929 42930 42931 42932 42933 42934 42935 42936 42937 42938 42939 42940 42941 42942 42943 42944 42945 42946 42947 42948 42949 42950 42951 42952 42953 42954 42955 42956 42957 42958 42959 42960 42961 42962 42963 42964 42965 42966 42967 42968 42969 42970 42971 42972 42973 42974 42975 42976 42977 42978 42979 42980 42981 42982 42983 42984 42985 42986 42987 42988 42989 42990 42991 42992 42993 42994 42995 42996 429

 44197 44198 44199 44200 44201 44202 44203 44204 44205 44206 44207 44208 44209 44210 44211 44212 44213 44214 44215 44216 44217 44218 44219 44220 44221 44222 44223 44224 44225 44226 44227 44228 44229 44230 44231 44232 44233 44234 44235 44236 44237 44238 44239 44240 44241 44242 44243 44244 44245 44246 44247 44248 44249 44250 44251 44252 44253 44254 44255 44256 44257 44258 44259 44260 44261 44262 44263 44264 44265 44266 44267 44268 44269 44270 44271 44272 44273 44274 44275 44276 44277 44278 44279 44280 44281 44282 44283 44284 44285 44286 44287 44288 44289 44290 44291 44292 44293 44294 44295 44296 44297 44298 44299 44300 44301 44302 44303 44304 44305 44306 44307 44308 44309 44310 44311 44312 44313 44314 44315 44316 44317 44318 44319 44320 44321 44322 44323 44324 44325 44326 44327 44328 44329 44330 44331 44332 44333 44334 44335 44336 44337 44338 44339 44340 44341 44342 44343 44344 44345 44346 44347 44348 44349 44350 44351 44352 44353 44354 44355 44356 44357 44358 44359 44360 44361 44362 443

 45563 45564 45565 45566 45567 45568 45569 45570 45571 45572 45573 45574 45575 45576 45577 45578 45579 45580 45581 45582 45583 45584 45585 45586 45587 45588 45589 45590 45591 45592 45593 45594 45595 45596 45597 45598 45599 45600 45601 45602 45603 45604 45605 45606 45607 45608 45609 45610 45611 45612 45613 45614 45615 45616 45617 45618 45619 45620 45621 45622 45623 45624 45625 45626 45627 45628 45629 45630 45631 45632 45633 45634 45635 45636 45637 45638 45639 45640 45641 45642 45643 45644 45645 45646 45647 45648 45649 45650 45651 45652 45653 45654 45655 45656 45657 45658 45659 45660 45661 45662 45663 45664 45665 45666 45667 45668 45669 45670 45671 45672 45673 45674 45675 45676 45677 45678 45679 45680 45681 45682 45683 45684 45685 45686 45687 45688 45689 45690 45691 45692 45693 45694 45695 45696 45697 45698 45699 45700 45701 45702 45703 45704 45705 45706 45707 45708 45709 45710 45711 45712 45713 45714 45715 45716 45717 45718 45719 45720 45721 45722 45723 45724 45725 45726 45727 45728 457

 46929 46930 46931 46932 46933 46934 46935 46936 46937 46938 46939 46940 46941 46942 46943 46944 46945 46946 46947 46948 46949 46950 46951 46952 46953 46954 46955 46956 46957 46958 46959 46960 46961 46962 46963 46964 46965 46966 46967 46968 46969 46970 46971 46972 46973 46974 46975 46976 46977 46978 46979 46980 46981 46982 46983 46984 46985 46986 46987 46988 46989 46990 46991 46992 46993 46994 46995 46996 46997 46998 46999 47000 47001 47002 47003 47004 47005 47006 47007 47008 47009 47010 47011 47012 47013 47014 47015 47016 47017 47018 47019 47020 47021 47022 47023 47024 47025 47026 47027 47028 47029 47030 47031 47032 47033 47034 47035 47036 47037 47038 47039 47040 47041 47042 47043 47044 47045 47046 47047 47048 47049 47050 47051 47052 47053 47054 47055 47056 47057 47058 47059 47060 47061 47062 47063 47064 47065 47066 47067 47068 47069 47070 47071 47072 47073 47074 47075 47076 47077 47078 47079 47080 47081 47082 47083 47084 47085 47086 47087 47088 47089 47090 47091 47092 47093 47094 470

 48295 48296 48297 48298 48299 48300 48301 48302 48303 48304 48305 48306 48307 48308 48309 48310 48311 48312 48313 48314 48315 48316 48317 48318 48319 48320 48321 48322 48323 48324 48325 48326 48327 48328 48329 48330 48331 48332 48333 48334 48335 48336 48337 48338 48339 48340 48341 48342 48343 48344 48345 48346 48347 48348 48349 48350 48351 48352 48353 48354 48355 48356 48357 48358 48359 48360 48361 48362 48363 48364 48365 48366 48367 48368 48369 48370 48371 48372 48373 48374 48375 48376 48377 48378 48379 48380 48381 48382 48383 48384 48385 48386 48387 48388 48389 48390 48391 48392 48393 48394 48395 48396 48397 48398 48399 48400 48401 48402 48403 48404 48405 48406 48407 48408 48409 48410 48411 48412 48413 48414 48415 48416 48417 48418 48419 48420 48421 48422 48423 48424 48425 48426 48427 48428 48429 48430 48431 48432 48433 48434 48435 48436 48437 48438 48439 48440 48441 48442 48443 48444 48445 48446 48447 48448 48449 48450 48451 48452 48453 48454 48455 48456 48457 48458 48459 48460 484

In [153]:
full_vec = full_vec.reshape((48895, -1))
full_vec.shape

(48895, 96)

In [156]:
X_full = pd.concat([X, pd.DataFrame(full_vec)], axis=1)

In [157]:
X_full.to_csv("X_full.csv")

In [161]:
X_train, X_test, y_train, y_test = train_test_split(X_full, y)

In [164]:
regressors = [lm, lasso, ridge, rf]
for reg in regressors:
    pipe = Pipeline([
        ('scale', scaler),
        ('pca', pca),
        ('reg', reg)
    ])
    pipe.fit(X_train, y_train)
    y_pred = pipe.predict(X_test)
    rmse = np.sqrt(mean_squared_error(y_test, y_pred))
    print('{},{}'.format(reg, rmse))

LinearRegression(copy_X=True, fit_intercept=True, n_jobs=None, normalize=False),203.58251740534135
Lasso(alpha=1.0, copy_X=True, fit_intercept=True, max_iter=1000,
      normalize=False, positive=False, precompute=False, random_state=None,
      selection='cyclic', tol=0.0001, warm_start=False),203.6363121584978
Ridge(alpha=1.0, copy_X=True, fit_intercept=True, max_iter=None,
      normalize=False, random_state=None, solver='auto', tol=0.001),203.58235141727013
RandomForestRegressor(bootstrap=True, criterion='mse', max_depth=None,
                      max_features='auto', max_leaf_nodes=None,
                      min_impurity_decrease=0.0, min_impurity_split=None,
                      min_samples_leaf=1, min_samples_split=2,
                      min_weight_fraction_leaf=0.0, n_estimators=10,
                      n_jobs=None, oob_score=False, random_state=None,
                      verbose=0, warm_start=False),232.58073940627224


In [165]:
# rf is slow and weak here, so we remove it.
# without pca
regressors = [lm, lasso, ridge]
for reg in regressors:
    pipe = Pipeline([
        ('scale', scaler),
        ('reg', reg)
    ])
    pipe.fit(X_train, y_train)
    y_pred = pipe.predict(X_test)
    rmse = np.sqrt(mean_squared_error(y_test, y_pred))
    print('{},{}'.format(reg, rmse))

LinearRegression(copy_X=True, fit_intercept=True, n_jobs=None, normalize=False),203.5829287298944
Lasso(alpha=1.0, copy_X=True, fit_intercept=True, max_iter=1000,
      normalize=False, positive=False, precompute=False, random_state=None,
      selection='cyclic', tol=0.0001, warm_start=False),203.6222003062972
Ridge(alpha=1.0, copy_X=True, fit_intercept=True, max_iter=None,
      normalize=False, random_state=None, solver='auto', tol=0.001),203.58235141727013


In [166]:
from sklearn.ensemble import GradientBoostingRegressor
gbdt = GradientBoostingRegressor()

In [168]:
regressors = [lm, lasso, ridge, gbdt]
for reg in regressors:
    pipe = Pipeline([
        ('scale', scaler),
        ('pca', pca),
        ('reg', reg)
    ])
    pipe.fit(X_train, y_train)
    y_pred = pipe.predict(X_test)
    rmse = np.sqrt(mean_squared_error(y_test, y_pred))
    print('{},{}'.format(reg, rmse))

LinearRegression(copy_X=True, fit_intercept=True, n_jobs=None, normalize=False),203.58251740534135
Lasso(alpha=1.0, copy_X=True, fit_intercept=True, max_iter=1000,
      normalize=False, positive=False, precompute=False, random_state=None,
      selection='cyclic', tol=0.0001, warm_start=False),203.6363121584978
Ridge(alpha=1.0, copy_X=True, fit_intercept=True, max_iter=None,
      normalize=False, random_state=None, solver='auto', tol=0.001),203.58235141727013
GradientBoostingRegressor(alpha=0.9, criterion='friedman_mse', init=None,
                          learning_rate=0.1, loss='ls', max_depth=3,
                          max_features=None, max_leaf_nodes=None,
                          min_impurity_decrease=0.0, min_impurity_split=None,
                          min_samples_leaf=1, min_samples_split=2,
                          min_weight_fraction_leaf=0.0, n_estimators=100,
                          n_iter_no_change=None, presort='auto',
                          random_state=No

## Model Optimization

In [169]:
# Lasso

In [181]:
pipe = Pipeline([
    ('scale', scaler),
    ('reg', lasso)
])

In [182]:
pipe.get_params().keys()

dict_keys(['memory', 'steps', 'verbose', 'scale', 'reg', 'scale__copy', 'scale__with_mean', 'scale__with_std', 'reg__alpha', 'reg__copy_X', 'reg__fit_intercept', 'reg__max_iter', 'reg__normalize', 'reg__positive', 'reg__precompute', 'reg__random_state', 'reg__selection', 'reg__tol', 'reg__warm_start'])

In [183]:
params = {'reg__alpha':[0.001,0.01,0.1,1]}

In [184]:
gsearch = GridSearchCV(pipe, param_grid=params, verbose=10)
gsearch.fit(X_train, y_train)

Fitting 3 folds for each of 4 candidates, totalling 12 fits


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.


[CV] reg__alpha=0.001 ................................................
[CV] .................... reg__alpha=0.001, score=0.100, total=  38.1s


[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:   38.0s remaining:    0.0s


[CV] reg__alpha=0.001 ................................................
[CV] .................... reg__alpha=0.001, score=0.102, total=  36.8s


...

[CV] reg__alpha=0.1 ..................................................
[CV] ...................... reg__alpha=0.1, score=0.117, total=   8.4s


[Parallel(n_jobs=1)]: Done   9 out of   9 | elapsed:  4.0min remaining:    0.0s


[CV] reg__alpha=1 ....................................................
[CV] ........................ reg__alpha=1, score=0.100, total=   1.9s
[CV] reg__alpha=1 ....................................................
[CV] ........................ reg__alpha=1, score=0.102, total=   1.9s
[CV] reg__alpha=1 ....................................................
[CV] ........................ reg__alpha=1, score=0.118, total=   2.2s


[Parallel(n_jobs=1)]: Done  12 out of  12 | elapsed:  4.1min finished


GridSearchCV(cv='warn', error_score='raise-deprecating',
             estimator=Pipeline(memory=None,
                                steps=[('scale',
                                        StandardScaler(copy=True,
                                                       with_mean=True,
                                                       with_std=True)),
                                       ('reg',
                                        Lasso(alpha=1.0, copy_X=True,
                                              fit_intercept=True, max_iter=1000,
                                              normalize=False, positive=False,
                                              precompute=False,
                                              random_state=None,
                                              selection='cyclic', tol=0.0001,
                                              warm_start=False))],
                                verbose=False),
             iid='warn', n_jobs=None,
   

In [188]:
y_pred = gsearch.best_estimator_.predict(X_full)
rmse = np.sqrt(mean_squared_error(y, y_pred))
print('{},{}'.format(gsearch.best_estimator_, rmse))

Pipeline(memory=None,
         steps=[('scale',
                 StandardScaler(copy=True, with_mean=True, with_std=True)),
                ('reg',
                 Lasso(alpha=0.1, copy_X=True, fit_intercept=True,
                       max_iter=1000, normalize=False, positive=False,
                       precompute=False, random_state=None, selection='cyclic',
                       tol=0.0001, warm_start=False))],
         verbose=False),225.8202205180784


In [193]:
y_pred[y_pred < 0] = 0
rmse = np.sqrt(mean_squared_error(y, y_pred))
print('{},{}'.format(gsearch.best_estimator_, rmse))

Pipeline(memory=None,
         steps=[('scale',
                 StandardScaler(copy=True, with_mean=True, with_std=True)),
                ('reg',
                 Lasso(alpha=0.1, copy_X=True, fit_intercept=True,
                       max_iter=1000, normalize=False, positive=False,
                       precompute=False, random_state=None, selection='cyclic',
                       tol=0.0001, warm_start=False))],
         verbose=False),225.72085416775136


In [196]:
y_pred

array([ 88.50013909, 253.23249259, 264.49388784, ..., 198.34810482,
       122.02516607, 155.36118405])

In [194]:
prediction = pd.DataFrame({'id':[]})
prediction['id'] = df['id']
prediction['price'] = y_pred

In [197]:
prediction.to_csv('best_prediction1.csv',index=False)

In [199]:
# the other submission posted
comp = pd.read_csv("for_comparison.csv").fillna(0)
np.sqrt(mean_squared_error(y, comp['price']))

228.38146715908672

In [176]:
# Ridge

In [177]:
pipe = Pipeline([
    ('scale', scaler),
    ('reg', ridge)
])

In [178]:
pipe.get_params().keys()

dict_keys(['memory', 'steps', 'verbose', 'scale', 'reg', 'scale__copy', 'scale__with_mean', 'scale__with_std', 'reg__alpha', 'reg__copy_X', 'reg__fit_intercept', 'reg__max_iter', 'reg__normalize', 'reg__random_state', 'reg__solver', 'reg__tol'])

In [179]:
params = {'reg__alpha':[0.001,0.01,0.1,1]}
gsearch = GridSearchCV(pipe, param_grid=params, verbose=10)
gsearch.fit(X_train, y_train)

Fitting 3 folds for each of 4 candidates, totalling 12 fits


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.


[CV] reg__alpha=0.001 ................................................
[CV] .................... reg__alpha=0.001, score=0.100, total=   0.3s


[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.2s remaining:    0.0s


[CV] reg__alpha=0.001 ................................................
[CV] .................... reg__alpha=0.001, score=0.102, total=   0.3s


...

[CV] reg__alpha=0.1 ..................................................
[CV] ...................... reg__alpha=0.1, score=0.117, total=   0.2s


[Parallel(n_jobs=1)]: Done   9 out of   9 | elapsed:    2.3s remaining:    0.0s


[CV] reg__alpha=1 ....................................................
[CV] ........................ reg__alpha=1, score=0.100, total=   0.2s
[CV] reg__alpha=1 ....................................................
[CV] ........................ reg__alpha=1, score=0.102, total=   0.3s
[CV] reg__alpha=1 ....................................................
[CV] ........................ reg__alpha=1, score=0.117, total=   0.3s


[Parallel(n_jobs=1)]: Done  12 out of  12 | elapsed:    3.1s finished


GridSearchCV(cv='warn', error_score='raise-deprecating',
             estimator=Pipeline(memory=None,
                                steps=[('scale',
                                        StandardScaler(copy=True,
                                                       with_mean=True,
                                                       with_std=True)),
                                       ('reg',
                                        Ridge(alpha=1.0, copy_X=True,
                                              fit_intercept=True, max_iter=None,
                                              normalize=False,
                                              random_state=None, solver='auto',
                                              tol=0.001))],
                                verbose=False),
             iid='warn', n_jobs=None,
             param_grid={'reg__alpha': [0.001, 0.01, 0.1, 1]},
             pre_dispatch='2*n_jobs', refit=True, return_train_score=False,
             s

In [180]:
y_pred = gsearch.best_estimator_.predict(X_test)
rmse = np.sqrt(mean_squared_error(y_test, y_pred))
print('{},{}'.format(gsearch.best_estimator_, rmse))

Pipeline(memory=None,
         steps=[('scale',
                 StandardScaler(copy=True, with_mean=True, with_std=True)),
                ('reg',
                 Ridge(alpha=1, copy_X=True, fit_intercept=True, max_iter=None,
                       normalize=False, random_state=None, solver='auto',
                       tol=0.001))],
         verbose=False),203.58235141727013


# To sum up, Lasso(alpha=0.1) is our best estimator.

# Next, we experiments with a (simple) DL model.

In [238]:
import keras
from keras import Sequential
from keras.layers import Dense

In [204]:
X_full.shape

(48895, 115)

In [241]:
# Hyperparameters for our network
input_size = 115
hidden_sizes = (512, 48895)
output_size = 1
# Build a feed-forward network
model = Sequential()
model.add(Dense(520, input_dim=115, activation='relu'))
model.add(Dense(256, activation='relu'))
model.add(Dense(1))
model.summary()

Model: "sequential_4"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_7 (Dense)              (None, 520)               60320     
_________________________________________________________________
dense_8 (Dense)              (None, 256)               133376    
_________________________________________________________________
dense_9 (Dense)              (None, 1)                 257       
Total params: 193,953
Trainable params: 193,953
Non-trainable params: 0
_________________________________________________________________


In [243]:
model.compile(loss='mean_squared_error', optimizer='adam', metrics=['mean_squared_error'])

In [245]:
model.fit(X_full, y, epochs=50, batch_size=10)

Epoch 1/50


19960/48895 [===========>..................] - ETA: 17:57 - loss: 22383.4102 - mean_squared_error: 22383.410 - ETA: 1:17 - loss: 16367.8216 - mean_squared_error: 16367.820 - ETA: 47s - loss: 13795.3601 - mean_squared_error: 13795.3604 - ETA: 36s - loss: 28880.9639 - mean_squared_error: 28880.968 - ETA: 32s - loss: 26401.2317 - mean_squared_error: 26401.234 - ETA: 28s - loss: 29711.3212 - mean_squared_error: 29711.324 - ETA: 26s - loss: 30198.5476 - mean_squared_error: 30198.548 - ETA: 25s - loss: 39296.3367 - mean_squared_error: 39296.339 - ETA: 24s - loss: 38191.8533 - mean_squared_error: 38191.851 - ETA: 23s - loss: 38020.9309 - mean_squared_error: 38020.933 - ETA: 21s - loss: 59523.5735 - mean_squared_error: 59523.574 - ETA: 21s - loss: 55724.5389 - mean_squared_error: 55724.527 - ETA: 20s - loss: 54695.1747 - mean_squared_error: 54695.164 - ETA: 20s - loss: 53222.1489 - mean_squared_error: 53222.132 - ETA: 19s - loss: 51370.2058 - mean_squared_error: 51370.203 - ETA: 19s - loss: 48

40850/48895 [========================>.....] - ETA: 10s - loss: 67881.2103 - mean_squared_error: 67881.164 - ETA: 10s - loss: 67440.1364 - mean_squared_error: 67440.093 - ETA: 10s - loss: 68429.9230 - mean_squared_error: 68429.882 - ETA: 9s - loss: 69201.1069 - mean_squared_error: 69201.070 - ETA: 9s - loss: 69096.7688 - mean_squared_error: 69096.72 - ETA: 9s - loss: 68638.0110 - mean_squared_error: 68637.96 - ETA: 9s - loss: 68696.7604 - mean_squared_error: 68696.71 - ETA: 9s - loss: 68362.9703 - mean_squared_error: 68362.92 - ETA: 9s - loss: 67933.1246 - mean_squared_error: 67933.08 - ETA: 9s - loss: 67793.5427 - mean_squared_error: 67793.50 - ETA: 9s - loss: 69172.8244 - mean_squared_error: 69172.79 - ETA: 9s - loss: 69070.6760 - mean_squared_error: 69070.65 - ETA: 9s - loss: 68615.1145 - mean_squared_error: 68615.09 - ETA: 9s - loss: 68337.0165 - mean_squared_error: 68336.99 - ETA: 9s - loss: 67921.7132 - mean_squared_error: 67921.70 - ETA: 9s - loss: 67547.8311 - mean_squared_erro

48895/48895 [==============================] - ETA: 2s - loss: 54297.6267 - mean_squared_error: 54297.64 - ETA: 2s - loss: 54482.4760 - mean_squared_error: 54482.51 - ETA: 2s - loss: 54318.4504 - mean_squared_error: 54318.49 - ETA: 2s - loss: 54157.2761 - mean_squared_error: 54157.31 - ETA: 2s - loss: 53977.7561 - mean_squared_error: 53977.79 - ETA: 2s - loss: 53832.7889 - mean_squared_error: 53832.83 - ETA: 2s - loss: 53670.2306 - mean_squared_error: 53670.26 - ETA: 2s - loss: 53587.4270 - mean_squared_error: 53587.46 - ETA: 2s - loss: 53561.2387 - mean_squared_error: 53561.29 - ETA: 2s - loss: 53379.2213 - mean_squared_error: 53379.27 - ETA: 2s - loss: 53592.5231 - mean_squared_error: 53592.58 - ETA: 2s - loss: 53485.3475 - mean_squared_error: 53485.41 - ETA: 2s - loss: 53473.8422 - mean_squared_error: 53473.89 - ETA: 2s - loss: 53306.7016 - mean_squared_error: 53306.75 - ETA: 1s - loss: 53189.7185 - mean_squared_error: 53189.77 - ETA: 1s - loss: 53087.0417 - mean_squared_error: 5308

20160/48895 [===========>..................] - ETA: 39s - loss: 34866.9766 - mean_squared_error: 34866.976 - ETA: 19s - loss: 120207.9472 - mean_squared_error: 120207.929 - ETA: 18s - loss: 67241.8398 - mean_squared_error: 67241.8438  - ETA: 17s - loss: 61215.6753 - mean_squared_error: 61215.679 - ETA: 17s - loss: 51012.5210 - mean_squared_error: 51012.523 - ETA: 17s - loss: 43134.7236 - mean_squared_error: 43134.730 - ETA: 17s - loss: 41244.9079 - mean_squared_error: 41244.914 - ETA: 16s - loss: 53725.4205 - mean_squared_error: 53725.418 - ETA: 16s - loss: 47838.2068 - mean_squared_error: 47838.207 - ETA: 16s - loss: 58691.0775 - mean_squared_error: 58691.070 - ETA: 16s - loss: 54626.8697 - mean_squared_error: 54626.859 - ETA: 16s - loss: 56379.8890 - mean_squared_error: 56379.886 - ETA: 16s - loss: 52231.1781 - mean_squared_error: 52231.183 - ETA: 16s - loss: 48852.5861 - mean_squared_error: 48852.589 - ETA: 16s - loss: 47958.9567 - mean_squared_error: 47958.953 - ETA: 16s - loss: 47

...

40600/48895 [=======================>......] - ETA: 9s - loss: 40737.2389 - mean_squared_error: 40737.26 - ETA: 9s - loss: 40483.3297 - mean_squared_error: 40483.35 - ETA: 9s - loss: 40528.7722 - mean_squared_error: 40528.80 - ETA: 9s - loss: 40375.9468 - mean_squared_error: 40375.97 - ETA: 9s - loss: 40143.8573 - mean_squared_error: 40143.88 - ETA: 9s - loss: 39940.6095 - mean_squared_error: 39940.64 - ETA: 9s - loss: 44228.1916 - mean_squared_error: 44228.22 - ETA: 9s - loss: 44040.7558 - mean_squared_error: 44040.78 - ETA: 9s - loss: 43798.3178 - mean_squared_error: 43798.35 - ETA: 9s - loss: 47848.6314 - mean_squared_error: 47848.66 - ETA: 8s - loss: 47860.4305 - mean_squared_error: 47860.46 - ETA: 8s - loss: 47563.6352 - mean_squared_error: 47563.67 - ETA: 8s - loss: 47333.3895 - mean_squared_error: 47333.43 - ETA: 8s - loss: 47951.9702 - mean_squared_error: 47952.00 - ETA: 8s - loss: 47688.8919 - mean_squared_error: 47688.94 - ETA: 8s - loss: 47436.5407 - mean_squared_error: 4743

48895/48895 [==============================] - ETA: 2s - loss: 46016.9298 - mean_squared_error: 46017.02 - ETA: 2s - loss: 45919.9945 - mean_squared_error: 45920.08 - ETA: 2s - loss: 45825.7455 - mean_squared_error: 45825.83 - ETA: 2s - loss: 45798.3023 - mean_squared_error: 45798.39 - ETA: 2s - loss: 45686.9389 - mean_squared_error: 45687.03 - ETA: 2s - loss: 45599.7295 - mean_squared_error: 45599.83 - ETA: 2s - loss: 45519.3216 - mean_squared_error: 45519.41 - ETA: 2s - loss: 47730.2360 - mean_squared_error: 47730.33 - ETA: 2s - loss: 47976.5038 - mean_squared_error: 47976.60 - ETA: 2s - loss: 47919.2047 - mean_squared_error: 47919.30 - ETA: 2s - loss: 47997.2559 - mean_squared_error: 47997.35 - ETA: 2s - loss: 47869.1058 - mean_squared_error: 47869.20 - ETA: 2s - loss: 47738.0544 - mean_squared_error: 47738.15 - ETA: 2s - loss: 47593.4735 - mean_squared_error: 47593.57 - ETA: 2s - loss: 47600.5848 - mean_squared_error: 47600.68 - ETA: 2s - loss: 47495.1758 - mean_squared_error: 4749

20060/48895 [===========>..................] - ETA: 39s - loss: 23588.8359 - mean_squared_error: 23588.835 - ETA: 17s - loss: 571221.0499 - mean_squared_error: 571221.062 - ETA: 16s - loss: 303214.4163 - mean_squared_error: 303214.437 - ETA: 16s - loss: 212241.9312 - mean_squared_error: 212241.906 - ETA: 16s - loss: 161506.3094 - mean_squared_error: 161506.281 - ETA: 16s - loss: 176120.9484 - mean_squared_error: 176120.921 - ETA: 16s - loss: 149428.8330 - mean_squared_error: 149428.781 - ETA: 16s - loss: 131204.4304 - mean_squared_error: 131204.375 - ETA: 16s - loss: 116920.4234 - mean_squared_error: 116920.359 - ETA: 16s - loss: 115906.9276 - mean_squared_error: 115906.882 - ETA: 16s - loss: 107758.0468 - mean_squared_error: 107757.976 - ETA: 16s - loss: 99749.8871 - mean_squared_error: 99749.8047  - ETA: 16s - loss: 93787.1343 - mean_squared_error: 93787.070 - ETA: 15s - loss: 86959.0655 - mean_squared_error: 86959.007 - ETA: 15s - loss: 81671.6292 - mean_squared_error: 81671.578 - E

40890/48895 [========================>.....] - ETA: 9s - loss: 34842.0804 - mean_squared_error: 34842.07 - ETA: 9s - loss: 34952.7293 - mean_squared_error: 34952.71 - ETA: 9s - loss: 34759.5339 - mean_squared_error: 34759.52 - ETA: 9s - loss: 34575.9009 - mean_squared_error: 34575.89 - ETA: 9s - loss: 34437.8703 - mean_squared_error: 34437.85 - ETA: 9s - loss: 34317.0198 - mean_squared_error: 34317.00 - ETA: 9s - loss: 36060.9803 - mean_squared_error: 36060.96 - ETA: 9s - loss: 35892.3891 - mean_squared_error: 35892.36 - ETA: 9s - loss: 37543.1216 - mean_squared_error: 37543.10 - ETA: 9s - loss: 37504.7814 - mean_squared_error: 37504.76 - ETA: 9s - loss: 37346.0197 - mean_squared_error: 37346.00 - ETA: 9s - loss: 37450.2634 - mean_squared_error: 37450.24 - ETA: 9s - loss: 37332.0935 - mean_squared_error: 37332.07 - ETA: 9s - loss: 37130.4907 - mean_squared_error: 37130.47 - ETA: 9s - loss: 36968.3223 - mean_squared_error: 36968.30 - ETA: 9s - loss: 36798.3318 - mean_squared_error: 3679

48895/48895 [==============================] - ETA: 2s - loss: 45333.9421 - mean_squared_error: 45333.93 - ETA: 2s - loss: 45301.8430 - mean_squared_error: 45301.83 - ETA: 2s - loss: 45154.3537 - mean_squared_error: 45154.34 - ETA: 2s - loss: 45026.1865 - mean_squared_error: 45026.17 - ETA: 2s - loss: 44928.1598 - mean_squared_error: 44928.14 - ETA: 2s - loss: 44801.0956 - mean_squared_error: 44801.08 - ETA: 2s - loss: 44677.3857 - mean_squared_error: 44677.37 - ETA: 2s - loss: 44561.8431 - mean_squared_error: 44561.83 - ETA: 2s - loss: 46728.9127 - mean_squared_error: 46728.90 - ETA: 2s - loss: 46598.8987 - mean_squared_error: 46598.89 - ETA: 2s - loss: 46471.4127 - mean_squared_error: 46471.41 - ETA: 2s - loss: 46372.4961 - mean_squared_error: 46372.50 - ETA: 2s - loss: 46227.0123 - mean_squared_error: 46227.02 - ETA: 1s - loss: 48364.7542 - mean_squared_error: 48364.76 - ETA: 1s - loss: 48272.2350 - mean_squared_error: 48272.24 - ETA: 1s - loss: 48148.6286 - mean_squared_error: 4814

In [246]:
y_pred_dl = model.predict(X_full)
rmse = np.sqrt(mean_squared_error(y, y_pred_dl))
rmse

211.28824271148818

In [247]:
prediction_dl = pd.DataFrame({'id':[]})
prediction_dl['id'] = df['id']
prediction_dl['price'] = y_pred_dl

In [248]:
prediction_dl.to_csv('prediction2.csv',index=False)